In [ ]:
import pandas as pd
import yaml
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

from IPython.display import Markdown, display
import visualizer_helpers as vh

In [ ]:
# NOTE: SPECIALLY TAGGED PARAMETERS CELL
# Assignments in this cell may be overwritten at compile-time
# Otherwise, the below defaults to not filtering any zones
zone_set: str = 'all'
how_method: str = 'any'
affected_tazs: list = []
affected_mazs: list = []

In [ ]:
# get corresponding filter from zone_set parameter
single_filter_tazs, multi_filter_tazs = vh.get_filters(zone_set, how_method, affected_tazs)
single_filter_mazs, multi_filter_mazs = vh.get_filters(zone_set, how_method, affected_mazs)

In [ ]:
# file IO locations
with open('_quarto.yml') as f:
    config = yaml.safe_load(f)
base_dir = config['sources']['base']
build_dir = config['sources']['build']

## Auto Ownership

In [ ]:
base_hh = pd.read_csv(f"{base_dir}/final_households.csv",
                       index_col='household_id', 
                       usecols=['household_id','auto_ownership','home_zone_id'])

build_hh = pd.read_csv(f"{build_dir}/final_households.csv", 
                       index_col='household_id', 
                       usecols=['household_id','auto_ownership','home_zone_id'])


In [ ]:
print(f"Filtering households by TAZ.\nOriginal HHs: {len(base_hh)}")
base_hh = base_hh[single_filter_mazs(base_hh.home_zone_id)]
build_hh = build_hh[single_filter_mazs(build_hh.home_zone_id)]
print(f"HHs after filtering: {len(base_hh)}")

In [ ]:
df = pd.crosstab(base_hh.auto_ownership, 
                 build_hh.auto_ownership, 
                 rownames=['Base'],
                 colnames=['Build'], 
                 margins=True, 
                 margins_name='Total')
df

## Work from Home

In [ ]:
base_per = pd.read_csv(f"{base_dir}/final_persons.csv",
                       index_col='person_id',
                       usecols=['person_id','work_from_home','home_zone_id'])

build_per = pd.read_csv(f"{build_dir}/final_persons.csv",
                        index_col='person_id',
                        usecols=['person_id','work_from_home','home_zone_id'])

In [ ]:
print(f"Filtering persons by home TAZ. Original Persons: {len(base_per)}")
base_per = base_per[single_filter_mazs(base_per.home_zone_id)]
build_per = build_per[single_filter_mazs(build_per.home_zone_id)]
print(f"Persons after filtering: {len(base_per)}")

In [ ]:
df = pd.crosstab(base_per.work_from_home, 
                 build_per.work_from_home, 
                 rownames=['Base'], 
                 colnames=['Build'], 
                 margins=True, 
                 margins_name='Total')
df

## School Location Changes

In [ ]:
base_per = pd.read_csv(f"{base_dir}/final_persons.csv",
                       index_col='person_id',
                       usecols=['person_id','is_student','school_zone_id','home_zone_id'])
build_per = pd.read_csv(f"{build_dir}/final_persons.csv",
                        index_col='person_id',
                        usecols=['person_id','is_student','school_zone_id','home_zone_id'])

In [ ]:
print(f"Filtering persons by home TAZ. Original Persons: {len(base_per)}")
base_per = base_per[single_filter_mazs(base_per.home_zone_id)]
build_per = build_per[single_filter_mazs(build_per.home_zone_id)]
print(f"Persons after filtering: {len(base_per)}")

In [ ]:
df = base_per[base_per.is_student].merge(build_per[build_per.is_student],
                                         how='left',
                                         left_index=True,
                                         right_index=True,
                                         suffixes=('_base','_build'))

df = (df.school_zone_id_base == df.school_zone_id_build).value_counts()
df.index = df.index.map({True:'Unchanged',False:'Changed'})
df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## Workplace Location Changes

In [ ]:
base_per = pd.read_csv(f"{base_dir}/final_persons.csv",
                       index_col='person_id',
                       usecols=['person_id','is_worker','workplace_zone_id','home_zone_id'])

build_per = pd.read_csv(f"{build_dir}/final_persons.csv",
                        index_col='person_id',
                        usecols=['person_id','is_worker','workplace_zone_id','home_zone_id'])

In [ ]:
print(f"Filtering persons by home TAZ. Original Persons: {len(base_per)}")
base_per = base_per[single_filter_mazs(base_per.home_zone_id)]
build_per = build_per[single_filter_mazs(build_per.home_zone_id)]
print(f"Persons after filtering: {len(base_per)}")

In [ ]:
df = base_per[base_per.is_worker].merge(build_per[build_per.is_worker],
                                        how='left',
                                        left_index=True,
                                        right_index=True, 
                                        suffixes=('_base','_build'))

df = (df.workplace_zone_id_base == df.workplace_zone_id_build).value_counts()
df.index = df.index.map({True:'Unchanged',False:'Changed'})
df

In [ ]:
fig = vh.create_pie_chart(df.to_frame().sort_index(),["count"])
fig.show()

## Transit Pass Subsidy

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## Transit Pass Ownership

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## Free Parking

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## Telecommute Frequency

In [ ]:
print("Lorem ipsum") # Placeholder for CS

## Transponder Ownership

In [ ]:
base_hh = pd.read_csv(f"{base_dir}/final_households.csv", 
                      index_col='household_id', 
                      usecols=['household_id','transponder_ownership','home_zone_id'])

build_hh = pd.read_csv(f"{build_dir}/final_households.csv", 
                       index_col='household_id', 
                       usecols=['household_id','transponder_ownership', 'home_zone_id'])


In [ ]:
print(f"Filtering households by TAZ. Original HHs: {len(base_hh)}")
base_hh = base_hh[single_filter_mazs(base_hh.home_zone_id)]
build_hh = build_hh[single_filter_mazs(build_hh.home_zone_id)]
print(f"HHs after filtering: {len(base_hh)}")

In [ ]:
df = pd.crosstab(base_hh.transponder_ownership,
                 build_hh.transponder_ownership, 
                 rownames=['Base'],
                 colnames=['Build'], 
                 margins=True, 
                 margins_name='Total')
df